In [1]:
!rm -rf minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-04 12:56:30--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-04 12:56:30 (96.3 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import requests

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

torch.random.manual_seed(0)

2024-07-04 12:56:37.628590: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-04 12:56:37.884585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 12:56:38.018192: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 12:56:38.019308: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 12:56:38.270302: I tensorflow/core/platform/cpu_feature_gua

In [7]:
# model_name = "microsoft/Phi-3-mini-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    "./models/phi-3/mini-model",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True
)


tokenizer = AutoTokenizer.from_pretrained("./models/phi-3/mini-tokenizer")

# Save the model and tokenizer
# model.save_pretrained("./models/Phi-3/mini-model")
# tokenizer.save_pretrained("./models/Phi-3/mini-tokenizer")

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [9]:
messages = [
    {"role": "user", "content": "What is the best learning path to become a data engineer?"},
]

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}
output = pipe(messages, **generation_args)
print(output[0]["generated_text"])

/home/teng/dev/myvenva/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


 Becoming a data engineer involves a combination of education, practical experience, and continuous learning. Here's a suggested learning path to help you become a data engineer:

1. Obtain a strong foundation in computer science and mathematics:
   - Complete a bachelor's degree in computer science, information technology, or a related field.
   - Take courses in mathematics, statistics, and algorithms to build a strong foundation in data analysis and problem-solving.

2. Learn programming languages:
   - Master programming languages such as Python, Java, and Scala, which are commonly used in data engineering.
   - Learn to use data manipulation and analysis libraries such as Pandas, NumPy, and Apache Spark.

3. Gain knowledge in data engineering tools and technologies:
   - Learn about data storage and management systems such as Hadoop, Apache Hive, and Apache HBase.
   - Familiarize yourself with data processing frameworks like Apache Spark, Apache Flink, and Apache Storm.
   - Unde

In [10]:
def build_prompt(query, search_results):
    prompt_template = """
        QUESTION: {question}

        CONTEXT: {context}
    """.strip()

    context_str = ""

    for doc in search_results:
        context_str = context_str + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context_str).strip()
    return prompt 

def llm(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]

    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }
    output = pipe(messages, **generation_args)
    return output[0]["generated_text"].strip() 

In [11]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [14]:
rag("I just discovered about the course. Can I still join?")

/home/teng/dev/myvenva/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


"QUESTION: I just discovered about the course. Can I still join?\n\nANSWER: Yes, you can still join the course even if you discover it after the start date. You are still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects. So, it's recommended not to leave everything for the last minute.\n\nTo join the course, you need to register before the course starts. You can register using the provided link. Additionally, you should subscribe to the course's public Google Calendar (available only on Desktop) and join the course's Telegram channel for announcements. Don't forget to register in DataTalks.Club's Slack and join the channel as well.\n\nIf you have any other questions or need further assistance, feel free to ask."